### Installing packages

In [ ]:
!pip install torch scikit-learn onnx onnx2torch onnxruntime z3-solver

### Fitting a neural network 

In [58]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import onnx

# fixing random seed
random_seed = 42
torch.manual_seed(random_seed)

# Load and preprocess the Iris dataset
iris = load_iris()  
X, y = iris.data, iris.target  

# Standardize the features
scaler = StandardScaler()  
X = scaler.fit_transform(X)  

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

# Convert data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# Define the neural network model
class IrisNet(nn.Module):
    def __init__(self):
        """
        Initializes the layers of the neural network.
        """
        super(IrisNet, self).__init__()
        self.fc1 = nn.Linear(4, 10)  # First fully connected layer (input: 4, output: 10)
        self.fc2 = nn.Linear(10, 10)  # Second fully connected layer (input: 10, output: 10)
        self.fc3 = nn.Linear(10, 3)  # Third fully connected layer (input: 10, output: 3)

    def forward(self, x):
        """
        Defines the forward pass of the network.
        
        Arguments:
        x -- input tensor
        
        Returns:
        Tensor containing the network's output after passing through the layers and activation functions.
        """
        x = F.relu(self.fc1(x))  # Apply ReLU activation to the output of the first layer
        x = F.relu(self.fc2(x))  # Apply ReLU activation to the output of the second layer
        x = self.fc3(x)  # Pass through the third layer (no activation)
        return x  # Return the final output

# Initialize the model, loss function, and optimizer
model = IrisNet()
criterion = nn.CrossEntropyLoss()  # Use cross-entropy loss for classification
optimizer = optim.Adam(model.parameters(), lr=1e-3)  # Use Adam optimizer with a learning rate of 0.001

# Train the network
epochs = 200
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 20 == 0:
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = (predicted == y_test).sum().item() / len(y_test)
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

    
# Export the model to ONNX format
dummy_input = torch.randn(1, 4)  # Create a dummy input with the same shape as the network's input

# Export the trained model to the ONNX format for use in other frameworks or tools
torch.onnx.export(model, dummy_input, "iris_model.onnx", input_names=['input'], output_names=['output'])


Epoch [20/200], Loss: 1.0888
Epoch [40/200], Loss: 1.0480
Epoch [60/200], Loss: 0.9925
Epoch [80/200], Loss: 0.9173
Epoch [100/200], Loss: 0.8261
Epoch [120/200], Loss: 0.7295
Epoch [140/200], Loss: 0.6387
Epoch [160/200], Loss: 0.5641
Epoch [180/200], Loss: 0.5079
Epoch [200/200], Loss: 0.4657
Test Accuracy: 83.33%


### Implementation of a neural network verifier

In [59]:
import torch
import onnx
from onnx2torch import convert
from z3 import *

# Load the ONNX model
onnx_model = onnx.load("iris_model.onnx")  # Load the ONNX model from file
model = convert(onnx_model)  # Convert the ONNX model to a PyTorch model


def get_weights_and_biases(model):
    """
    Extracts the weights and biases from the converted PyTorch model.
    
    Arguments:
    model -- The converted PyTorch model.
    
    Returns:
    A tuple containing the weights and biases for each layer of the model.
    """
    # Access the layers and extract weights and biases
    weights_1 = model._modules['fc1/Gemm'].weight.detach().numpy()
    bias_1 = model._modules['fc1/Gemm'].bias.detach().numpy()
    weights_2 = model._modules['fc2/Gemm'].weight.detach().numpy()
    bias_2 = model._modules['fc2/Gemm'].bias.detach().numpy()
    weights_3 = model._modules['fc3/Gemm'].weight.detach().numpy()
    bias_3 = model._modules['fc3/Gemm'].bias.detach().numpy()
    
    return (weights_1, bias_1), (weights_2, bias_2), (weights_3, bias_3)


def ReLU(x):
    """
    Implements the ReLU activation function for Z3.
    
    Arguments:
    x -- Input value or list of values.
    
    Returns:
    The ReLU-transformed value(s).
    """
    if isinstance(x, list):
        return [If(element > 0, element, 0) for element in x]
    else:
        return If(x > 0, x, 0)

    
def Linear(x, weights, bias):
    """
    Implements the linear layer computation (Wx + b) for Z3.
    
    Arguments:
    x -- Input list of Z3 Real variables.
    weights -- Weights matrix for the layer.
    bias -- Bias vector for the layer.
    
    Returns:
    The output list after applying the linear transformation.
    """
    output = [sum(x[i] * weights[j][i] for i in range(len(x))) + bias[j] for j in range(len(weights))]
    return output


def encode_network_in_z3(solver, model, input_var):
    """
    Encodes the neural network layers in Z3 using the provided weights, biases, and input variables.
    
    Arguments:
    solver -- Z3 solver instance.
    model -- The converted PyTorch model.
    input_var -- List of Z3 Real variables representing the input.
    
    Returns:
    The final output of the network encoded in Z3.
    """
    # Get weights and biases
    (weights_1, bias_1), (weights_2, bias_2), (weights_3, bias_3) = get_weights_and_biases(model)
    
    # Encode each layer using Z3 operations
    x = ReLU(Linear(input_var, weights_1, bias_1))
    x = ReLU(Linear(x, weights_2, bias_2))
    x = Linear(x, weights_3, bias_3)
    return x

def add_precondition(solver, data_point, epsilon):
    """
    Adds the precondition to the solver ensuring that the input variables are within an epsilon neighborhood of the original data point.
    
    Arguments:
    solver -- Z3 solver instance.
    data_point -- The original data point.
    epsilon -- The maximum allowed perturbation (epsilon).
    """
    for i in range(len(data_point)):
        solver.add(Real(f'x{i}') >= data_point[i] - epsilon)
        solver.add(Real(f'x{i}') <= data_point[i] + epsilon)

def add_postcondition1(solver, encoded_output, original_class):
    """
    Adds the postcondition to the solver ensuring that the predicted class does not change.
    
    Arguments:
    solver -- Z3 solver instance.
    encoded_output -- The output of the network encoded in Z3.
    original_class -- The original predicted class.
    """
    for i in range(len(encoded_output)):
        if i != original_class:
            solver.add(encoded_output[original_class] > encoded_output[i])
            
def add_postcondition(solver, encoded_output, original_class):
    """
    Adds the negation of the postcondition to the solver ensuring that the predicted class does not change.
    
    Arguments:
    solver -- Z3 solver instance.
    encoded_output -- The output of the network encoded in Z3.
    original_class -- The original predicted class.
    
    Returns:
    None
    """
    conditions = []
    for i in range(len(encoded_output)):
        if i != original_class:
            conditions.append(encoded_output[original_class] <= encoded_output[i])
    solver.add(Or(conditions))


def verify_robustness(model, data_point, epsilon):
    """
    Verifies the robustness of the model for a given data point and epsilon.
    
    Arguments:
    model -- The converted PyTorch model.
    data_point -- The original data point.
    epsilon -- The maximum allowed perturbation (epsilon).
    
    Returns:
    "Counterexample" and the counterexample if a counterexample is found (i.e., the network is not robust),
    "Verified" if the network is robust for the given epsilon.
    """
    solver = Solver()
    
    # Convert data point to Z3 Real variables
    input_vars = [Real(f'x{i}') for i in range(len(data_point))]

    # Add the precondition (input bounds)
    add_precondition(solver, data_point, epsilon)

    # Encode the network in Z3
    encoded_output = encode_network_in_z3(solver, model, input_vars)

    # Determine the original class
    original_class = torch.argmax(model(torch.tensor(data_point, dtype=torch.float32))).item()

    # Add the postcondition (output class should not change)
    add_postcondition(solver, encoded_output, original_class)

    # Check the constraints
    if solver.check() == sat:
        return "Counterexample", solver.model()
    else:
        return "Verified"

### Verify the neural network you trained in Step 1. Use the first data point from the IRIS dataset and

In [60]:
data_point = X[0]  
epsilon_values = [0.1, 2]  
for epsilon in epsilon_values:
    result = verify_robustness(model, data_point, epsilon)
    print(f'Epsilon = {epsilon}: {result}')  

Epsilon = 0.1: Verified
Epsilon = 2: ('Counterexample', [x1 = -24567327825276391577302539972952625239036631859242110360732007484582700883641858763974423221/27504752150924431874051297421688514497314073881745377480215685150295579809121187500000000000,
 x3 = 989417005377131239913195165619894742315765943719345764489512627490515150384741938458884351419/1760304137659163639939283034988064927828100728431704158733803849618917107783756000000000000000,
 x0 = 109931882970219/100000000000000,
 x2 = -70511633757113757829048261111358663686089211677684518249866750422731021818406839609487363621/51773651107622459998214206911413674347885315542108945845111877929968150228934000000000000000])


So for Epsilon = 0.1 model is robust, while for Epsilon = 2 counterexample found. Now let's iteratively find maximum (with a certain error associated with discretization) Epsilon for which model is still robust. We will use binary search algorithm for that.

In [61]:
def print_max_robust_eps(model, data_point, start=0.1, end=2.0, tolerance=0.0001):
    """
    Finds and prints the maximum epsilon value for which the model is still robust using binary search.
    
    Arguments:
    model -- The converted PyTorch model.
    data_point -- The original data point.
    start -- The starting epsilon value for the search (default is 0.1).
    end -- The ending epsilon value for the search (default is 2.0).
    tolerance -- The stopping criterion for the binary search (default is 0.0001).
    
    Returns:
    The maximum epsilon value for which the model is still robust.
    """
    max_verified_epsilon = 0  # Initialize the maximum verified epsilon

    while abs(end - start) >= tolerance:
        mid = (start + end) / 2  # Calculate the midpoint
        result = verify_robustness(model, data_point, mid)

        if result == "Verified":
            max_verified_epsilon = mid
            start = mid + tolerance  # Move the start to mid + tolerance to search the higher range
        else:
            end = mid - tolerance  # Move the end to mid - tolerance to search the lower range

    if max_verified_epsilon == 0:
        print("The model is not verified even for the minimal epsilon.")
    else:
        print(f"Epsilon = {max_verified_epsilon:.4f} is the maximal (with tolerance={tolerance}) epsilon value for which the model is still robust.")

data_point = X[0]
print_max_robust_eps(model, data_point, start=0.1, end=2.0)

Epsilon = 1.0952 is the maximal (with tolerance=0.0001) epsilon value for which the model is still robust.
